# View OEE Diário

Cálculo de OEE (Overall Equipment Effectiveness) diário.
OEE = Disponibilidade x Performance x Qualidade

In [ ]:
# Configuração (lê parâmetros do job)
try:
    CATALOG = dbutils.widgets.get('catalog')
except:
    CATALOG = 'manufatura_lakehouse'

try:
    SCHEMA_SILVER = dbutils.widgets.get('schema_silver')
except:
    SCHEMA_SILVER = 'silver'

try:
    SCHEMA_GOLD = dbutils.widgets.get('schema_gold')
except:
    SCHEMA_GOLD = 'gold'

def fqtn(schema, table):
    if CATALOG and CATALOG.lower() != 'none':
        return f"`{CATALOG}`.`{schema}`.`{table}`"
    else:
        return f"`{schema}`.`{table}`"

# Criar schema se necessário
if CATALOG and CATALOG.lower() != 'none':
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{CATALOG}`.`{SCHEMA_GOLD}`")
    spark.sql(f"USE CATALOG `{CATALOG}`")
else:
    spark.sql(f"CREATE DATABASE IF NOT EXISTS `{SCHEMA_GOLD}`")


In [ ]:
spark.sql(f"""
CREATE OR REPLACE VIEW {fqtn(SCHEMA_GOLD, 'vw_oee_diario')} AS
WITH production_daily AS (
    SELECT 
        equipment_id, 
        start_date_key,
        sum(planned_qty) as total_planned,
        sum(actual_qty) as total_produced,
        sum(duration_minutes) as production_minutes
    FROM {fqtn(SCHEMA_GOLD, 'fact_producao')}
    GROUP BY 1, 2
),
downtime_daily AS (
    SELECT 
        equipment_id, 
        start_date_key,
        sum(downtime_minutes) as total_downtime
    FROM {fqtn(SCHEMA_GOLD, 'fact_manutencao')}
    GROUP BY 1, 2
),
quality_daily AS (
    SELECT
        equipment_id,
        date_key,
        sum(total_quantity) as total_inspected,
        sum(failed_quantity) as total_failed
    FROM {fqtn(SCHEMA_GOLD, 'fact_qualidade')}
    GROUP BY 1, 2
)
SELECT 
    p.start_date_key,
    t.full_date,
    p.equipment_id,
    e.equipment_name,
    coalesce(d.total_downtime, 0) as downtime_min,
    p.production_minutes,
    (p.production_minutes + coalesce(d.total_downtime, 0)) as total_time,
    round(p.production_minutes / (p.production_minutes + coalesce(d.total_downtime, 0.01)), 4) as availability_rate,
    round(p.total_produced / nullif(p.total_planned, 0), 4) as performance_rate,
    round((q.total_inspected - q.total_failed) / nullif(q.total_inspected, 0), 4) as quality_rate
FROM production_daily p
JOIN {fqtn(SCHEMA_GOLD, 'dim_tempo')} t ON p.start_date_key = t.date_key
JOIN {fqtn(SCHEMA_GOLD, 'dim_equipamento')} e ON p.equipment_id = e.equipment_id
LEFT JOIN downtime_daily d ON p.equipment_id = d.equipment_id AND p.start_date_key = d.start_date_key
LEFT JOIN quality_daily q ON p.equipment_id = q.equipment_id AND p.start_date_key = q.date_key
""")